as_04: Asylum applications from main applicants and dependants, by age, sex and country of nationality

In [1]:
def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')

if is_interactive():
    %run lib/scrape_govuk.ipynb
    metadata = scrape('https://www.gov.uk/government/statistics/immigration-statistics-october-to-december-2017-data-tables')
    import pyexcel
    from io import BytesIO

    ods_files = [f for f in metadata['files']
                 if f['type'] == 'ODS' and
                 f['title'] == 'Asylum data tables immigration statistics October to December 2017 volume 2 second edition']
    assert len(ods_files) == 1, 'Should be exactly one matching ODS file'

    ods_url = ods_files[0]['url']
    ods_title = ods_files[0]['title']

    ods_file = BytesIO(session.get(ods_files[0]['url']).content)

    data = pyexcel.get_sheet(file_content=ods_file, file_type='ods', sheet_name='as_04', library='pyexcel-ods3')

https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/691964/asylum2-oct-dec-2017-tables.ods

In [2]:
df = pd.DataFrame(data.get_array())
df

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,Table as_04: Asylum applications from main app...,,,,,,,,,,...,,,,,,,,,,
1,Back to contents,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,,,,,,,,
3,Year,Geographical region,Country of nationality,Total Applications,Total male applications,Male: \nunder 5,Male: 5-9,Male: 10-13,Male: 14-15,Male: 16-17,...,Female: 21-24,Female: 25-29,Female: 30-34,Female: 35-39,Female: 40-49,Female: 50-59,Female: 60-64,Female: \n65+,Female: Age unknown,Sex unknown
4,2008,*Total,*Total,31313,21020,1017,521,657,1118,1940,...,1103,2054,1433,962,927,362,97,143,159,30
5,2008,Africa North,*Total Africa North,883,757,24,13,6,15,44,...,10,28,15,9,11,4,0,4,6,1
6,2008,Africa Sub-Saharan,*Total Africa Sub-Saharan,11852,6067,556,199,134,101,287,...,594,1205,860,545,530,219,50,69,68,7
7,2008,America North,*Total America North,26,13,0,0,0,1,1,...,0,2,1,1,0,0,0,1,0,0
8,2008,America Central and South,*Total America Central and South,477,311,18,10,6,4,5,...,12,28,34,19,25,6,0,0,0,0
9,2008,Asia Central,*Total Asia Central,3776,3524,37,28,210,600,756,...,23,49,13,14,21,6,9,9,2,4


In [3]:
observations = df.loc[3:, :36]
observations.rename(columns= observations.iloc[0], inplace=True)
observations.drop(observations.index[0], inplace = True)
observations.drop('Geographical region', axis = 1, inplace = True)
Final_table = pd.melt(observations,
                       ['Year','Country of nationality'],
                       var_name= 'Application',
                       value_name="Value")
Final_table.Value.dropna(inplace =True)
Final_table.rename(columns={'Country of nationality': 'Nationality'}, inplace=True)
Final_table['Unit'] = 'Applications'
Final_table['Measure Type'] = 'Count'

In [4]:
Final_table['Value'] = Final_table['Value'].map(lambda x : ''
                                                   if (x == '.') | (x == 'z') | ( x == ':')
                                                    else x )

In [5]:
import numpy as np
Final_table['Value'].replace('', np.nan, inplace=True)
Final_table.dropna(subset=['Value'], inplace=True)
Final_table['Value'] = Final_table['Value'].apply(lambda x: pd.to_numeric(x, downcast='integer'))
Final_table['Value'] = Final_table['Value'].astype(int)

In [6]:
Final_table.rename(columns={'Year': 'Period'}, inplace=True)
Final_table['Period'] = Final_table['Period'].astype(str)
Final_table = Final_table[Final_table['Period'] != '']
Final_table['Period'] = 'year/' + Final_table['Period'].map(lambda cell: cell.replace(' ','-'))

In [7]:
Final_table['Nationality'] = Final_table['Nationality'].str.lstrip('*')

In [8]:
Final_table['Nationality'] = Final_table['Nationality'].map(
    lambda x: {
        'Total Africa North' : 'Africa North',
        'Total Africa Sub-Saharan' : 'Africa Sub-Saharan',
        'Total America North' : 'America North',
        'Total America Central and South ' : 'America Central and South',
        'Total Asia Central' : 'Asia Central',
        'Total Asia East' : 'Asia East',
        'Total Asia South' : 'Asia South',
        'Total Asia South East' : 'Asia South East',
        'Total EU 14' : 'EU 14',
        'Total EU 2' : 'EU 2',
        'Total EU 8' : 'EU 8',
        'Total EU Other' : 'EU Other',
        'Total Europe Other' : 'Europe Other',
        'Total Middle East' : 'Middle East',
        'Total Oceania' : 'Oceania',
        'Total Other' : 'Other'                    
        }.get(x, x))

In [9]:
import urllib.request as request
import csv
import io
import requests
r = request.urlopen('https://raw.githubusercontent.com/ONS-OpenData/ref_migration/master/codelists/ho-country-of-nationality.csv').read().decode('utf8').split("\n")
reader = csv.reader(r)
url="https://raw.githubusercontent.com/ONS-OpenData/ref_migration/master/codelists/ho-country-of-nationality.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
Final_table = pd.merge(Final_table, c, how = 'left', left_on = 'Nationality', right_on = 'Label')
Final_table.columns = ['Country of nationality' if x=='Notation' else x for x in Final_table.columns]

In [10]:
Final_table = Final_table[['Country of nationality','Period','Application','Measure Type','Value','Unit']]

In [11]:
Final_table['Age'] = Final_table['Application'].map(
    lambda x: {
        'Total Applications' : 'all',
        'Total male applications' : 'all', 
        'Male: \nunder 5' : 'hoad/under-5',
         'Male: 5-9' : 'hoad/5-9', 
        'Male: 10-13' : 'hoad/10-13', 
        'Male: 14-15' : 'hoad/14-15', 
        'Male: 16-17' : 'hoad/16-17',
         'Male: 18-20' : 'hoad/18-20', 
        'Male: 21-24' : 'hoad/21-24', 
        'Male: 25-29' : 'hoad/25-29', 
        'Male: 30-34' : 'hoad/30-34',
         'Male: 35-39' : 'hoad/35-39', 
        'Male: 40-49' : 'hoad/40-49', 
        'Male: 50-59': 'hoad/50-59', 
        'Male: 60-64' : 'hoad/60-64',
        'Male: 65+' : 'hoad/65-plus', 
        'Male: Age unknown' : 'hoad/unknown', 
        'Total female applications' : 'all',
        'Female: under 5' : 'hoad/under-5', 
        'Female: \n5-9' : 'hoad/5-9', 
        'Female: 10-13' : 'hoad/10-13',
         'Female: 14-15' : 'hoad/14-15', 
        'Female: 16-17' : 'hoad/16-17', 
        'Female: 18-20' : 'hoad/18-20', 
        'Female: 21-24': 'hoad/21-24',
        'Female: 25-29' : 'hoad/25-29', 
        'Female: 30-34': 'hoad/30-34', 
        'Female: 35-39' : 'hoad/35-39', 
        'Female: 40-49' : 'hoad/40-49',
         'Female: 50-59' : 'hoad/50-59', 
        'Female: 60-64' : 'hoad/60-64', 
        'Female: \n65+': 'hoad/65-plus',
        'Female: Age unknown' : 'hoad/unknown', 
        'Sex unknown' : 'hoad/unknown'
        }.get(x, x))

In [12]:
Final_table['Sex'] = Final_table['Application'].map(
    lambda x: {
        'Total Applications' : 'T',
        'Total male applications' : 'M', 
        'Male: \nunder 5' : 'M',
         'Male: 5-9' : 'M', 
        'Male: 10-13' : 'M', 
        'Male: 14-15' : 'M', 
        'Male: 16-17' : 'M',
         'Male: 18-20' : 'M', 
        'Male: 21-24' : 'M', 
        'Male: 25-29' : 'M', 
        'Male: 30-34' : 'M',
         'Male: 35-39' : 'M', 
        'Male: 40-49' : 'M', 
        'Male: 50-59': 'M', 
        'Male: 60-64' : 'M',
        'Male: 65+' : 'M', 
        'Male: Age unknown' : 'M', 
        'Total female applications' : 'F',
        'Female: under 5' : 'F', 
        'Female: \n5-9' : 'F', 
        'Female: 10-13' : 'F',
         'Female: 14-15' : 'F', 
        'Female: 16-17' : 'F', 
        'Female: 18-20' : 'F', 
        'Female: 21-24': 'F',
        'Female: 25-29' : 'F', 
        'Female: 30-34': 'F', 
        'Female: 35-39' : 'F', 
        'Female: 40-49' : 'F',
         'Female: 50-59' : 'F', 
        'Female: 60-64' : 'F', 
        'Female: \n65+': 'F',
        'Female: Age unknown' : 'F', 
        'Sex unknown' : 'U'
        }.get(x, x))

In [13]:
Final_table = Final_table[['Country of nationality','Period','Age','Sex','Measure Type','Value','Unit']]

In [14]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

Final_table.to_csv(destinationFolder / ('as_04.csv'), index = False)